In [73]:
import pandas as pd
import datetime
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib as plt
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn import tree
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

Para correr la carga de los files y el filtro de personas correr abajo 

In [83]:
personas_entrenar = pd.read_csv('data/labels_training_set.csv', low_memory=False)
informacion = pd.read_csv('data/events_up_to_01062018.csv', low_memory= False)
personas_clasificar = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory = False)

info = personas_entrenar.merge(informacion, on = 'person', how = 'inner')
info = info[['person']]
info['frecuencia'] = 1
data = pd.DataFrame({'frecuencia': info.groupby('person').aggregate(sum)['frecuencia']}).reset_index()



personas_entrenar_frec = personas_entrenar.merge(data, on = 'person', how = 'left')
# personas antes de filtrar 
print(personas_entrenar_frec.count())
print(personas_entrenar_frec.label.value_counts())

personas_entrenar_frec= personas_entrenar_frec[((personas_entrenar_frec['label'] == 1)) | ((personas_entrenar_frec['frecuencia'] < 260) & (personas_entrenar_frec['label'] == 0))]

print(personas_entrenar_frec.count())
print(personas_entrenar_frec.label.value_counts())
personas_entrenar=personas_entrenar_frec


person        19414
label         19414
frecuencia    19414
dtype: int64
0    18434
1      980
Name: label, dtype: int64
person        18738
label         18738
frecuencia    18738
dtype: int64
0    17758
1      980
Name: label, dtype: int64


In [84]:
def preprocesar_info(data, features, separacion):
    """funcion que prepocesa el training set
    
    data es el dataframe completo que le saco los features 
    
    separacion es para separar train de validation 
    
    """
    data = data.fillna(0)
    data_dic = data[features]  
    
    
    data_set_list = data_dic.values.tolist()
    dic = {}
    dic['0'] = 0
    dic_inverso = {}
    i = 32
    
    for lista in data_set_list:
        for element in lista:
            if element not in dic:
                    dic[element]= i
                    dic_inverso[str(i)] = element
                    i+=1
            
                
    data[features] = data[features].applymap(lambda x: dic[str(x)])
    data= data[['timestamp','label','sku'] + features]
    data['timestamp'] = pd.to_datetime(data.timestamp, format = '%Y-%m-%d %H:%M:%S')
    data["dia"] = data['timestamp'].apply(lambda x: x.day)
    
    del data['timestamp']
    
    
    for x in range(5):
        data= data.sample(frac=1)

    Y = np.array(data['label'])
   
    del data ['label']
    
    
    
    
    largo = int(len(data.index)*separacion)
    
    X = data.values
    
    #scaler = StandardScaler()
    #scaler.fit(X)
    #X = scaler.transform(X)
    
    train_x = X[:largo]
    train_y = Y[:largo]
    validation_x = X[largo:]
    validation_y = Y[largo:]

    return train_x, train_y, validation_x, validation_y, dic

In [86]:
def preprocesar_predic(data, features, dic):
    """
    preprocesa el set a predecir
    aca lo mismo pero que para el train pero sin el label
    recibe el dic viejo con los valores que use en el train.
    
    si el valor no esta en el dic que viene de train lo agrega.
    
    esto pasa porque saco a muchos que tienen cero en label por lo que sus 
    atributos no entran al dic antes
    
    i arranca en 1000 para que tenga peso que es un atributo que viene de label 0 
    
    """
    data = data.fillna(0)
    data_to_dic = data[features]
    
    data_set_list = data_to_dic.values.tolist()
       
    #nota de color:     
    i = 1000
    
    for lista in data_set_list:
        for element in lista:
            if element not in dic:
                    dic[element]= i
                    i+=1
            
               

    data[features] = data[features].applymap(lambda x: dic[str(x)])
    data = data[['timestamp','sku'] + features] 
    data['timestamp'] = pd.to_datetime(data.timestamp, format = '%Y-%m-%d %H:%M:%S')
    data["dia"] = data['timestamp'].apply(lambda x: x.day)
  
    del data['timestamp']
    X = data.values
    
    #scaler = StandardScaler()
    #scaler.fit(X)
    #X = scaler.transform(X)

    return X

Aca hace el train de ambos data set y los predice

guarda el resultado en data 

In [87]:
train_data = personas_entrenar.merge(informacion, on = 'person', how = 'inner')

#aca podes elegir los features 
features = ['event', 'model','condition','storage','color','city','region','country','new_vs_returning','device_type','channel', 'operating_system_version', 'browser_version' ]

train_x, train_y, validation_x, validation_y,dic = preprocesar_info(train_data, features,0.75)

#imprimo cantidad de 1s y 0s de los validation 
print(np.count_nonzero(validation_y == 1))
print(np.count_nonzero(validation_y == 0))

clf = tree.DecisionTreeClassifier()

clf.fit(train_x, train_y) 

pred = clf.predict(validation_x)

print(np.count_nonzero(pred == 1))
print(np.count_nonzero(pred == 0))


print ("# " + "score" + ": " + str(accuracy_score(validation_y, pred)))
print("fin del training")



prediccion_set = personas_clasificar.merge(informacion, on = 'person', how = 'inner')
inputs = preprocesar_predic(prediccion_set, features, dic)
prediccion_set['label'] =  clf.predict(inputs)



result = personas_clasificar.merge(prediccion_set, how='inner', on='person')
result = result[['person', 'label']]
result['label'] = result['label'].astype('int')
result = result.groupby('person').sum()
result.reset_index(inplace=True)
result['label'] = result['label'].apply(lambda x: 1 if x > 0 else 0)
print(result.label.value_counts())
result.to_csv('data/Decision_tree5.csv', encoding='utf-8', index=False)


25064
181524
5805
200783
# score: 0.8774711019033051
fin del training
0    12602
1     6813
Name: label, dtype: int64


esto no lo llegue a probar

In [ ]:
for i in range(1, 20):    
    knn = KNeighborsClassifier(n_neighbors=i)
    print(np.count_nonzero(validation_y == 1))
    print(np.count_nonzero(validation_y == 0))
    knn.fit(train_x, train_y)                   
    pred = knn.predict(validation_x)
    print(np.count_nonzero(pred == 1))
    print(np.count_nonzero(pred == 0))
    print ("# " + str(i) + ": " + str(accuracy_score(validation_y, pred)))

25064
181524
24168
182420
# 1: 0.817472457257924
25064
181524
5710
200878
# 2: 0.8748233198443278
25064
181524
11073
195515
# 3: 0.8644403353534571
25064
181524
4940
201648
# 4: 0.8783762851666118
25064
181524
7551
199037
# 5: 0.8739278176854416
25064
181524
4107
202481
# 6: 0.8799204213216644
25064
181524
5998
200590
# 7: 0.8764884698046354
25064
181524
3454
203134
# 8: 0.8801382461711232
25064
181524
5402
201186
# 9: 0.876653048579782
25064
181524
3350
203238
# 10: 0.8797025964722055
25064
181524
4149
202439
# 11: 0.8786715588514338
25064
181524
2849
203739
# 12: 0.8798236102774605
25064
181524
3451
203137
# 13: 0.8789619919840456
25064
181524
2426
204162
# 14: 0.8803705926772126
25064
181524
2899
203689
# 15: 0.8797267992332566
25064
181524
